In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import seaborn as sns
import pandas as pd
from scipy import stats
from scipy.optimize import curve_fit
from scipy.integrate import odeint
from scipy.integrate import solve_ivp

# These equations are based on Mumby et al. (2007) with the Grazing rate from McManus et al., 2023

## Parameters

In [2]:
#Define a function to calculate the derivative
r = 0.2 #coral growth rate
a = 0.3 #algal overgrowth onto corals
# mu = 0.04 #coral background mortality
gamma = 0.7 #algal growth rate
g0 = 0.2 #minimum grazing rate
g1 = 1 #maximum grazing rate



time_steps = 1000 #no. of time steps
start = 0 
stop = 1000
time_points = np.linspace(start, stop, time_steps+1)

In [9]:
def dNdt(C,M,P):
    
    dt = P['dt']
    mu = P['mu']


    #! Calculate the derivative
    gM = (g0 + (g1 - g0)*(C**0.5))*M

    dC = (r*C*(1-C-M) - a*C*M - mu*C) * dt #Coral equation
    dM = (gamma*M*(1-C-M) + a*C*M - gM) * dt #Macroalgae equation

    return dC, dM

### 4th order Runge-Kutta method

In [4]:
def RK4(C, M, P): #4th-order Runge-Kutta
    # Initial values
    C_init = C
    M_init = M

    # Step 1
    k1_C, k1_M = dNdt(C, M, P)

    # Step 2
    C1 = C + 0.5 * k1_C
    M1 = M + 0.5 * k1_M
    
    k2_C, k2_M= dNdt(C1, M1, P)

    # Step 3
    C2 = C + 0.5 * k2_C
    M2 = M + 0.5 * k2_M

    k3_C, k3_M = dNdt(C2, M2, P)

    # Step 4
    C3 = C + k3_C
    M3 = M + k3_M
    k4_C, k4_M = dNdt(C3, M3, P)

    # Calculate weighted average
    dC = (k1_C + 2 * k2_C + 2 * k3_C + k4_C) / 6
    dM = (k1_M + 2 * k2_M + 2 * k3_M + k4_M) / 6


    # Update values
    C = C_init + dC
    M = M_init + dM

    return C, M

In [5]:
def run_model(INIT_C, INIT_M, P):
   
    NUMYEARS = P['NUMYEARS']
    
    C = np.zeros((NUMYEARS+1))
    M = np.zeros((NUMYEARS+1))
    
    C[0] = INIT_C
    M[0] = INIT_M
    
    for year in np.arange(0,NUMYEARS):
        C[year+1],M[year+1] = RK4(C[year],M[year],P)
    
    return C, M

In [6]:
dt = 1 # time step (year)
NUMSTEPS = 1000
NUMYEARS = int(NUMSTEPS/dt)

# set initial conditions for C0 and M0
C0 = 0.2
M0 = 0.2


In [10]:
muSteps= 0.005
mu0 = 0.01
muMax = 0.2

# Create a dictionary object
parameterDictionary = {'dt':dt,
                    'NUMSTEPS': NUMSTEPS, 
                    'NUMYEARS': NUMYEARS, 
                    'r': r, 
                    'a': a,
                    'mu': muMax,
                    'gamma': gamma,
                    'g0': g0,
                    'g1': g1
                    }


muPerms = int((muMax-mu0)/muSteps)
simulations = pd.DataFrame({ 'mu' : [],
                            'c0' : [],
                            'm0' : [],
                            'cEnd' : [],
                            'mEnd' : []})

for t in np.linspace(mu0, muMax, muPerms):
    parameterDictionary['mu'] = t
    for c in [0.01, 0.05, 0.45, 0.75]:
        for m in [0.01, 0.05, 0.45, 0.75]:
            
            C_array, M_array = run_model(c, m, parameterDictionary)
            dfTemp = pd.DataFrame({
                                'mu' : [t],
                                'c0' : [c],
                                'm0' : [m],
                                'cEnd' : [C_array[-1]],
                                'mEnd' : [M_array[-1]]})

            simulations = pd.concat([simulations, dfTemp], ignore_index = True)
                                    
                                                

# df = pd.concat(simulations, ignore_index=True)
simulations.to_csv('mumbyIncreasingMortality.csv')